# Pipeline

In [1]:
# Reload modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

# Import from code root directory
import sys; sys.path.insert(0, '../')

# Imports
from access.amazon_doc2vec_reader import AmoreDoctovecReader
from selection.bounding_boxes import BoundingBoxes

# Reader

In [2]:
amazon_doc2vec_reader = AmoreDoctovecReader()
options = {}
options['data_directory'] = '/home/eml4u/EML4U/data/amazon-complete'
options['distributions_file'] = '/tmp/amore_test_1.pickle'
amazon_doc2vec_reader.initialize(options)

AmoreDoctovecReader distributions file: /tmp/amore_test_1.pickle
AmoreDoctovecReader data directory:     /home/eml4u/EML4U/data/amazon-complete


# Detector

In [3]:
bounding_boxes = BoundingBoxes()
options = {}
options['percentile'] = 0.2
options['enclosed_dimensions'] = 50
bounding_boxes.initialize(options)
bounding_boxes.set_reader(amazon_doc2vec_reader)

BoundingBoxes percentile:                            0.2
BoundingBoxes minimum number of enclosed dimensions: 50


In [4]:
print(len(bounding_boxes.select_item_ids(amazon_doc2vec_reader.get_distribution_ids()[0], amazon_doc2vec_reader.get_distribution_ids()[1])))

AmoreDoctovecReader: Loading distribution file: /tmp/amore_test_1.pickle
BoundingBoxes: Collecting values of single dimensions
AmazonPickleReader: Reading raw data
AmazonPickleReader: Reading bow50 data
BoundingBoxes: Updating boundaries
BoundingBoxes: Filtering items
6580
